# HMM (Hidden Markov Model) Basic

# 1 理论基础

HMM是概率图模型的一种。

**HMM模型可用三个变量描述**

* 状态转移概率矩阵
* 观测概率矩阵
* 初始状态概率向量

**HMM中存在的三大问题**

* 给定模型，求其生成观测序列的概率
* 给定模型和观测序列，求与该观测序列最匹配的状态序列
* 给定观测序列，求模型

# 2 sklearn中的HMM

[参考资料](http://hmmlearn.readthedocs.io/en/stable/tutorial.html)

sklearn提供了3中HMM模型
* GaussianHMM
* MultinomialHMM
* GMMHMM

# 3 实际使用

## 3.1 用参数生成一个GaussianHMM模型

### 3.1.1 法1
生成标准GaussianHMM，需要给定4个参数

In [2]:
import numpy as np
from hmmlearn import hmm
np.random.seed(42)

m = hmm.GaussianHMM(n_components=3, covariance_type="full")#3个状态

m.startprob_ = np.array([0.6, 0.3, 0.1])#初始状态概率向量（三大参数之1）
m.transmat_ = np.array([[0.7, 0.2, 0.1],#状态转移概率矩阵（三大参数之2）
                            [0.3, 0.5, 0.2],
                            [0.3, 0.3, 0.4]])

m.means_ = np.array([[0.0, 0.0], [3.0, -3.0], [5.0, 10.0]])#观测状态均值
m.covars_ = np.tile(np.identity(2), (3, 1, 1))#观测状态协方差
# 至此，模型生成完毕

# 下面用该模型生成100个观测值，与对应的状态值
X, Z = m.sample(100)


### 3.1.2 法2

生成left-right HMM，只需要给定2个参数

In [3]:
lrm = hmm.GaussianHMM(n_components=3, covariance_type="diag",
                      init_params="cm", params="cmt")
lrm.startprob_ = np.array([1.0, 0.0, 0.0])
lrm.transmat_ = np.array([[0.5, 0.5, 0.0],
                          [0.0, 0.5, 0.5],
                          [0.0, 0.0, 1.0]])


### 3.1.3 法3
用EM算法，可以做参数估计，只需要给定1个参数

In [4]:
m = hmm.GaussianHMM(n_components=3, n_iter=100, init_params="mcs")
m.transmat_ = np.array([[0.7, 0.2, 0.1],
                             [0.3, 0.5, 0.2],
                             [0.3, 0.3, 0.4]])


## 3.2 给定观测序列，求模型 (3大问题之1)

In [5]:
X = [[0], [1], [0], [1], [0], [0], [1], [0]]

m = hmm.GaussianHMM(n_components=3, covariance_type="full", n_iter=100)
m.fit(X)  


GaussianHMM(algorithm='viterbi', covariance_type='full', covars_prior=0.01,
      covars_weight=1, init_params='stmc', means_prior=0, means_weight=0,
      min_covar=0.001, n_components=3, n_iter=100, params='stmc',
      random_state=None, startprob_prior=1.0, tol=0.01, transmat_prior=1.0,
      verbose=False)

##  3.3 给定模型和观测序列，求与该观测序列最匹配的状态序列 (3大问题之2)

In [6]:
X = [[0], [1], [0]]
Z = m.predict(X)
Z

array([1, 0, 1])

## 3.3 给定模型，求其生成观测序列的概率(3大问题之3)

In [11]:
p = m.score([[0], [0], [1]])
p

3.9425601220046569